In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import yfinance as yf
import math
import torch.nn.functional as F

In [2]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        padding = 1 if torch.__version__ >= '1.5.0' else 2
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model,
                                   kernel_size=3, padding=padding, padding_mode='circular', bias=False)
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_in', nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x


class FixedEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(FixedEmbedding, self).__init__()

        w = torch.zeros(c_in, d_model).float()
        w.require_grad = False

        position = torch.arange(0, c_in).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        w[:, 0::2] = torch.sin(position * div_term)
        w[:, 1::2] = torch.cos(position * div_term)

        self.emb = nn.Embedding(c_in, d_model)
        self.emb.weight = nn.Parameter(w, requires_grad=False)

    def forward(self, x):
        return self.emb(x).detach()


class TemporalEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='fixed', freq='h'):
        super(TemporalEmbedding, self).__init__()

        minute_size = 4
        hour_size = 24
        weekday_size = 7
        day_size = 32
        month_size = 13

        Embed = FixedEmbedding if embed_type == 'fixed' else nn.Embedding
        if freq == 't':
            self.minute_embed = Embed(minute_size, d_model)
        self.hour_embed = Embed(hour_size, d_model)
        self.weekday_embed = Embed(weekday_size, d_model)
        self.day_embed = Embed(day_size, d_model)
        self.month_embed = Embed(month_size, d_model)

    def forward(self, x):
        x = x.long()
        minute_x = self.minute_embed(x[:, :, 4]) if hasattr(
            self, 'minute_embed') else 0.
        hour_x = self.hour_embed(x[:, :, 3])
        weekday_x = self.weekday_embed(x[:, :, 2])
        day_x = self.day_embed(x[:, :, 1])
        month_x = self.month_embed(x[:, :, 0])

        return hour_x + weekday_x + day_x + month_x + minute_x


class TimeFeatureEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='timeF', freq='h'):
        super(TimeFeatureEmbedding, self).__init__()

        freq_map = {'h': 4, 't': 5, 's': 6,
                    'm': 1, 'a': 1, 'w': 2, 'd': 3, 'b': 3}
        d_inp = freq_map[freq]
        self.embed = nn.Linear(d_inp, d_model, bias=False)

    def forward(self, x):
        return self.embed(x)


class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type=embed_type,
                                                    freq=freq) if embed_type != 'timeF' else TimeFeatureEmbedding(
            d_model=d_model, embed_type=embed_type, freq=freq)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        if x_mark is None:
            x = self.value_embedding(x) + self.position_embedding(x)
        else:
            # THE FIX IS HERE: We need to provide x_mark to use temporal_embedding
            x = self.value_embedding(
                x) + self.temporal_embedding(x_mark) + self.position_embedding(x)
        return self.dropout(x)

class Inception_Block_V1(nn.Module):
    def __init__(self, in_channels, out_channels, num_kernels=6, init_weight=True):
        super(Inception_Block_V1, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_kernels = num_kernels
        kernels = []
        for i in range(self.num_kernels):
            kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=2 * i + 1, padding=i))
        self.kernels = nn.ModuleList(kernels)
        if init_weight:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        res_list = []
        for i in range(self.num_kernels):
            res_list.append(self.kernels[i](x))
        res = torch.cat(res_list, dim=1)
        return res

def FFT_for_Period(x, k=2):
    # [B, T, C]
    xf = torch.fft.rfft(x, dim=1)
    # find period by amplitudes
    frequency_list = abs(xf).mean(0).mean(-1)
    frequency_list[0] = 0
    _, top_list = torch.topk(frequency_list, k)
    top_list = top_list.detach().cpu().numpy()
    period = x.shape[1] // top_list
    return period, abs(xf).mean(-1)[:, top_list]


class TimesBlock(nn.Module):
    def __init__(self, configs):
        super(TimesBlock, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.k = configs.top_k
        # parameter-efficient design
        self.conv = nn.Sequential(
            Inception_Block_V1(configs.d_model, configs.d_ff,
                               num_kernels=configs.num_kernels),
            nn.GELU(),
            Inception_Block_V1(configs.d_ff * configs.num_kernels, configs.d_model,
                               num_kernels=configs.num_kernels)
        )

    def forward(self, x):
        B, T, N = x.size()
        period_list, period_weight = FFT_for_Period(x, self.k)

        res = []
        for i in range(self.k):
            period = period_list[i]
            # padding
            if (self.seq_len + self.pred_len) % period != 0:
                length = (
                                 ((self.seq_len + self.pred_len) // period) + 1) * period
                padding = torch.zeros([x.shape[0], (length - (self.seq_len + self.pred_len)), x.shape[2]]).to(x.device)
                out = torch.cat([x, padding], dim=1)
            else:
                length = (self.seq_len + self.pred_len)
                out = x
            # reshape
            out = out.reshape(B, length // period, period,
                              N).permute(0, 3, 1, 2).contiguous()
            # 2D conv: from 1d Variation to 2d Variation
            out = self.conv(out)
            # reshape back
            out = out.permute(0, 2, 3, 1).reshape(B, -1, N)
            res.append(out[:, :(self.seq_len + self.pred_len), :])
        res = torch.stack(res, dim=-1)
        # adaptive aggregation
        period_weight = F.softmax(period_weight, dim=1)
        period_weight = period_weight.unsqueeze(
            1).unsqueeze(1).repeat(1, T, N, 1)
        res = torch.sum(res * period_weight, -1)
        # residual connection
        res = res + x
        return res


class Model(nn.Module):
    """
    Paper link: https://openreview.net/pdf?id=ju_Uqw384Oq
    """

    def __init__(self, configs):
        super(Model, self).__init__()
        self.configs = configs
        self.task_name = configs.task_name
        self.seq_len = configs.seq_len
        self.label_len = configs.label_len
        self.pred_len = configs.pred_len
        self.model = nn.ModuleList([TimesBlock(configs)
                                    for _ in range(configs.e_layers)])
        self.enc_embedding = DataEmbedding(configs.enc_in, configs.d_model, configs.embed, configs.freq,
                                           configs.dropout)
        self.layer = configs.e_layers
        self.layer_norm = nn.LayerNorm(configs.d_model)
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            self.predict_linear = nn.Linear(
                self.seq_len, self.pred_len + self.seq_len)
            self.projection = nn.Linear(
                configs.d_model, configs.c_out, bias=True)
        # ... (rest of the Model class)
    def forecast(self, x_enc, x_mark_enc, x_dec, x_mark_dec):
        # Normalization from Non-stationary Transformer
        means = x_enc.mean(1, keepdim=True).detach()
        x_enc = x_enc.sub(means)
        stdev = torch.sqrt(
            torch.var(x_enc, dim=1, keepdim=True, unbiased=False) + 1e-5)
        x_enc = x_enc.div(stdev)

        # embedding
        enc_out = self.enc_embedding(x_enc, x_mark_enc)  # [B,T,C]
        enc_out = self.predict_linear(enc_out.permute(0, 2, 1)).permute(
            0, 2, 1)  # align temporal dimension
        # TimesNet
        for i in range(self.layer):
            enc_out = self.layer_norm(self.model[i](enc_out))
        # project back
        dec_out = self.projection(enc_out)

        # De-Normalization from Non-stationary Transformer
        dec_out = dec_out.mul(stdev)
        dec_out = dec_out.add(means)
        return dec_out

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, mask=None):
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            dec_out = self.forecast(x_enc, x_mark_enc, x_dec, x_mark_dec)
            return dec_out[:, -self.pred_len:, :]  # [B, L, D]
        # ... (rest of the forward method)
        return None

In [3]:
def create_time_features(df, time_col='Date', freq='d'):
    """
    Creates time series features from a datetime index.
    """
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col])
    df['month'] = df[time_col].dt.month
    df['day'] = df[time_col].dt.day
    df['weekday'] = df[time_col].dt.dayofweek
    # Add more features if needed, like 'weekofyear'

    # The TimeFeatureEmbedding expects these features in a specific order based on `freq`
    if freq == 'd': # Daily
        feature_cols = ['month', 'day', 'weekday']
    else: # Add more logic for other frequencies if needed
        raise NotImplementedError(f"Frequency '{freq}' not implemented.")

    return df[feature_cols].values

# This function creates input/output sequences
def create_sequences(input_data, time_features, seq_len, pred_len):
    X, y, X_mark = [], [], []
    for i in range(len(input_data) - seq_len - pred_len + 1):
        X.append(input_data[i:(i + seq_len)])
        y.append(input_data[i + seq_len:i + seq_len + pred_len])
        X_mark.append(time_features[i:(i + seq_len)])
    return np.array(X), np.array(y), np.array(X_mark)

In [4]:
# Load Data
ticker = 'BTC-USD'
btc_data = yf.download(ticker, start='2014-01-01', end='2024-01-01')
btc_data.reset_index(inplace=True) # Move 'Date' from index to a column

# Feature Engineering
# 'Close' price - forecasting
price_data = btc_data[['Close']].values
# Create time features
time_features = create_time_features(btc_data, time_col='Date', freq='d')

# Scaling
scaler = StandardScaler()
# Fit only on the training data to avoid data leakage
train_size = int(len(price_data) * 0.8)
scaler.fit(price_data[:train_size])
scaled_data = scaler.transform(price_data)

# Create Sequences
configs = type("Configs", (), {})() # Create a mock configs class
configs.seq_len = 30 # Use 30 days of history
configs.pred_len = 1 # Predict 1 day ahead
configs.enc_in = 1 # We are using 1 feature (Close price)
configs.c_out = 1

X, y, X_mark = create_sequences(scaled_data, time_features, configs.seq_len, configs.pred_len)


/tmp/ipython-input-2504293634.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  btc_data = yf.download(ticker, start='2014-01-01', end='2024-01-01')
[*********************100%***********************]  1 of 1 completed


In [5]:
# Split Data
X_train, X_test, y_train, y_test, X_mark_train, X_mark_test = train_test_split(
    X, y, X_mark, test_size=0.2, shuffle=False # Time series data should not be shuffled
)

# to Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_mark_train = torch.tensor(X_mark_train, dtype=torch.float32)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
X_mark_test = torch.tensor(X_mark_test, dtype=torch.float32)

In [6]:
X_train.shape

torch.Size([2690, 30, 1])

In [7]:
#  3. Model Training
# Hyperparameters for the Model
configs.task_name = 'long_term_forecast'
configs.label_len = 0
configs.e_layers = 1
configs.d_model = 32
configs.d_ff = 64
configs.num_kernels = 3
configs.embed = 'timeF' # Use TimeFeatureEmbedding
configs.freq = 'd'
configs.dropout = 0.05
configs.top_k = 3

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize Model and Optimizer
model = Model(configs).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # Slightly increased learning rate

Using device: cuda


In [ ]:
# Training Loop
epochs = 200
epoch = 0
batch_size = 2690
best_loss = float('inf')
best_model_path = f'models/best_timesnet_model_{epoch}.pth'

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for i in range(0, len(X_train), batch_size):
        # Get batch and move to device
        batch_X = X_train[i:i+batch_size].to(device)
        batch_y = y_train[i:i+batch_size].to(device)
        batch_X_mark = X_mark_train[i:i+batch_size].to(device)

        # Zeros for decoder inputs, not used in this forecasting model
        batch_dec_input = torch.zeros_like(batch_y).to(device) # Placeholder
        batch_dec_mark = torch.zeros_like(batch_y).to(device) # Placeholder

        optimizer.zero_grad()

        outputs = model(batch_X, batch_X_mark, batch_dec_input, batch_dec_mark)

        # Output shape: [batch_size, pred_len, features]
        # Target shape: [batch_size, pred_len, features]
        loss = criterion(outputs, batch_y)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_epoch_loss = epoch_loss / (len(X_train) / batch_size)
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_epoch_loss:.6f}')

    # Save the best model
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        torch.save(model.state_dict(), best_model_path)
        print(f'Saved best model with loss: {best_loss:.6f}')

Epoch [1/200], Loss: 0.054949
Saved best model with loss: 0.054949
Epoch [2/200], Loss: 0.173463
Epoch [3/200], Loss: 0.079957
Epoch [4/200], Loss: 0.032963
Saved best model with loss: 0.032963
Epoch [5/200], Loss: 0.023362
Saved best model with loss: 0.023362
Epoch [6/200], Loss: 0.026210
Epoch [7/200], Loss: 0.028789
Epoch [8/200], Loss: 0.028183
Epoch [9/200], Loss: 0.026223
Epoch [10/200], Loss: 0.024501
Epoch [11/200], Loss: 0.023616
Epoch [12/200], Loss: 0.023360
Saved best model with loss: 0.023360
Epoch [13/200], Loss: 0.023509
Epoch [14/200], Loss: 0.023763
Epoch [15/200], Loss: 0.023959
Epoch [16/200], Loss: 0.023923
Epoch [17/200], Loss: 0.023667
Epoch [18/200], Loss: 0.023217
Saved best model with loss: 0.023217
Epoch [19/200], Loss: 0.022776
Saved best model with loss: 0.022776
Epoch [20/200], Loss: 0.022175
Saved best model with loss: 0.022175
Epoch [21/200], Loss: 0.021917
Saved best model with loss: 0.021917
Epoch [22/200], Loss: 0.021817
Saved best model with loss: 0.0

In [ ]:
# Load the best model
model.load_state_dict(torch.load(best_model_path))
model.eval() # Set the model to evaluation mode

In [ ]:
# Evaluation
with torch.no_grad():
    test_loss = 0
    # Use the full test dataset for evaluation
    batch_X_test = X_test.to(device)
    batch_y_test = y_test.to(device)
    batch_X_mark_test = X_mark_test.to(device)

    # Zeros for decoder inputs, not used in this forecasting model
    batch_dec_input_test = torch.zeros_like(batch_y_test).to(device) # Placeholder
    batch_dec_mark_test = torch.zeros_like(batch_y_test).to(device) # Placeholder

    outputs_test = model(batch_X_test, batch_X_mark_test, batch_dec_input_test, batch_dec_mark_test)

    loss_test = criterion(outputs_test, batch_y_test)
    test_loss = loss_test.item()

print(f'Test Loss: {test_loss:.6f}')

# Optional: Inverse transform the predictions and actual values for plotting or further analysis
predicted = outputs_test.cpu().numpy()
actual = batch_y_test.cpu().numpy()

# Reshape for inverse transform: (n_samples * pred_len, n_features)
predicted_reshaped = predicted.reshape(-1, predicted.shape[-1])
actual_reshaped = actual.reshape(-1, actual.shape[-1])

predicted_original_scale = scaler.inverse_transform(predicted_reshaped)
actual_original_scale = scaler.inverse_transform(actual_reshaped)

# Reshape back to original prediction shape: (n_samples, pred_len, n_features)
predicted_original_scale = predicted_original_scale.reshape(predicted.shape)
actual_original_scale = actual_original_scale.reshape(actual.shape)

print("Predictions in original scale (first 5):")
print(predicted_original_scale[:5])
print("\nActual values in original scale (first 5):")
print(actual_original_scale[:5])

# Plotting
import matplotlib.pyplot as plt

# Since pred_len is 1, we can plot the predicted vs actual points directly
# For pred_len > 1, you might need to adjust how you visualize the sequences

plt.figure(figsize=(12, 6))
plt.plot(actual_original_scale.flatten(), label='Actual')
plt.plot(predicted_original_scale.flatten(), label='Predicted')
plt.title('Bitcoin Price Prediction (Test Set)')
plt.xlabel('Time (days)')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()